## Imports

In [ ]:
import pandas as pd
from typing import Dict, List, Optional

import matplotlib.pyplot as plt
import seaborn as sns

from metrics.english_metrics import get_features
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix

## Getting Data from Wikipedia

In [ ]:
from utils.wikipedia_helper import get_text_from_wikipedia
from preprocessing.indic_preprocessor import preprocess_text
from translation.indic_en_translation import translate_indic_to_english

In [ ]:
lang_code = "mr"
text = get_text_from_wikipedia("विराट कोहली", lang_code)
sentences = preprocess_text(text, "mar_Deva")

# Translate sentences to English in batches
result = []
BATCH_SIZE = 10
for i in range(0, len(sentences), BATCH_SIZE):
    batch_sentences = sentences[i:i + BATCH_SIZE]
    result.extend(translate_indic_to_english([{"sentence": sentence} for sentence in batch_sentences], lang_code))

In [ ]:
# Save the results into csv
translated_df = pd.DataFrame(result)
translated_df.to_csv("models\\wikipedia_data.csv", index=False)

## Loading Wikipedia Data and getting features

In [ ]:
CSV_FILE = "models\\wikipedia_data.csv"
data = pd.read_csv(CSV_FILE)

detailed_data = get_features(data)

In [ ]:
# Export the detailed data to csv
detailed_data.to_csv("models\\final_detailed_data.csv", index=False)

## Active Learning on partial annotation

In [ ]:
# Load the partial detailed data
partial_annotated_data = pd.read_csv("models\\partial_annotation.csv")
partial_annotated_data = get_features(partial_annotated_data)

# Function to plot feature vs difficulty (scatter plot and line)
def plot_feature_vs_difficulty(feature_name: str):
    plt.figure(figsize=(12, 5))

    # Scatter plot
    plt.subplot(1, 2, 1)
    sns.scatterplot(x=partial_annotated_data[feature_name], y=partial_annotated_data['difficulty'])
    plt.title(f'Scatter Plot of {feature_name} vs Difficulty')
    plt.xlabel(feature_name)
    plt.ylabel('difficulty')

    # Line plot
    plt.subplot(1, 2, 2)
    sns.lineplot(x=partial_annotated_data[feature_name], y=partial_annotated_data['difficulty'])
    plt.title(f'Line Plot of {feature_name} vs Difficulty')
    plt.xlabel(feature_name)
    plt.ylabel('difficulty')

    plt.tight_layout()
    plt.show()
    
# Keep different features on x axis and difficulty on y axis
# features_to_plot = ['is_passive', 'subordinate_clause_count', 'noun_phrase_count', 'average_noun_phrase_length', 'prepositional_phrase_count', 'verb_phrase_count', 'adjective_count', 'adverb_count', 'proper_noun_count', 'sentence_length', 'avg_rh_score']
# for feature in features_to_plot:
#     # Line graph and scatter plot side by side
#     plot_feature_vs_difficulty(feature)

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from modAL.models import ActiveLearner
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# --- CUSTOM QUERY STRATEGY ---
def random_forest_variance_sampling(learner, X_pool):
    all_preds = np.array([
        tree.predict(X_pool) for tree in learner.estimator.estimators_
    ])
    variances = np.std(all_preds, axis=0)
    query_idx = np.argmax(variances)
    return np.array([query_idx]), X_pool[query_idx].reshape(1, -1)


# ----- PREPARE DATA -----
labeled_data = partial_annotated_data[partial_annotated_data['difficulty'].notnull()].copy()
unlabeled_data = partial_annotated_data[partial_annotated_data['difficulty'].isnull()].copy()

X_initial = labeled_data.drop(columns=['difficulty', 'sentence', 'translation']).values
y_initial = labeled_data['difficulty'].values

X_pool = unlabeled_data.drop(columns=['difficulty', 'sentence', 'translation']).values


# ----- ACTIVE LEARNER -----
learner = ActiveLearner(
    estimator=RandomForestRegressor(n_estimators=50),
    X_training=X_initial,
    y_training=y_initial,
    query_strategy=random_forest_variance_sampling
)


# ----- AUTOSAVE FUNCTION -----
def save_progress():
    """Save current progress back into a single CSV."""
    combined = pd.concat([labeled_data, unlabeled_data], ignore_index=True)
    combined.to_csv("models\\annotated_data.csv", index=False)


# ----- ACTIVE LEARNING LOOP -----
while len(unlabeled_data) > 0:

    # Query next sample
    query_idx, query_instance = learner.query(X_pool)
    q = query_idx[0]

    print(f"\n📊 Progress: Labeled = {len(labeled_data)}, Unlabeled = {len(unlabeled_data)}, Total = {len(labeled_data) + len(unlabeled_data)}")

    print("\nPlease annotate the following sentence:")
    print(unlabeled_data.iloc[q]['sentence'])
    print("Translation:")
    print(unlabeled_data.iloc[q]['translation'])

    # Get user input
    user_input = input("Enter difficulty score (1-10) or 'q' to quit: ").strip()

    if user_input.lower() == 'q':
        print("\n🔹 Quitting early...")
        save_progress()
        break
    elif not user_input.replace('.', '', 1).isdigit() or not (1 <= float(user_input) <= 10):
        print("❗ Invalid input. Please enter a number between 1 and 10.")
        continue

    difficulty = float(user_input)

    # Teach the model
    learner.teach(X_pool[q].reshape(1, -1), [difficulty])

    # Store annotation
    unlabeled_data.loc[q, "difficulty"] = difficulty

    # Move annotated row to labeled_data
    labeled_data = pd.concat([labeled_data, unlabeled_data.loc[[q]]], ignore_index=True)

    # Remove from active pool
    X_pool = np.delete(X_pool, q, axis=0)
    unlabeled_data = unlabeled_data.drop(unlabeled_data.index[q]).reset_index(drop=True)

else:
    # Runs only if loop finishes normally (no break)
    print("\n🎉 Completed all annotation steps!")
    save_progress()


In [ ]:
import joblib

# Save ONLY the Random Forest model
joblib.dump(learner.estimator, 'models\\al_random_forest_model.pkl')
print("Model saved successfully!")

## Trying Other Models

Step 1: Start the server
    : mlflow server --port 8080

Step 2: set tracking uri and experiment

Step 3: visualize

In [ ]:
import mlflow

# Connect to remote MLflow server
# Later shift these values to env
mlflow.set_tracking_uri("http://localhost:8080")
mlflow.set_experiment("my-genai-experiment")
print("✓ MLflow server configured.")

In [ ]:
# Splitting data

annotated_data = pd.read_csv("models\\annotated_data.csv")

from sklearn.model_selection import train_test_split

X = annotated_data.drop(columns=['difficulty', 'sentence', 'translation'])
y = annotated_data['difficulty']

print(y.min(), y.max())
# num_bins = 4
# bins = np.linspace(y.min(), y.max(), num_bins + 1)

# y_binned = np.digitize(y, bins) - 1  # Bins are 0-indexed
y_binned = pd.qcut(y, q=10, duplicates='drop') 

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y_binned, random_state=42)



In [ ]:
from sklearn.ensemble import RandomForestRegressor
with mlflow.start_run():

    # ✅ Log model parameters
    mlflow.log_param("model_type", "RandomForestRegressor")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)

    # ✅ Train model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    # ✅ Predictions
    rf_y_pred = rf_model.predict(X_test)

    # ✅ -------- PLOT 1: Predicted vs Actual --------
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, rf_y_pred)
    plt.plot([y_test.min(), y_test.max()],
             [y_test.min(), y_test.max()],
             'r--')
    plt.xlabel("Actual difficulty")
    plt.ylabel("Predicted difficulty")
    plt.title("Predicted vs Actual difficulty")

    plt.savefig("pred_vs_actual.png")
    mlflow.log_artifact("pred_vs_actual.png")
    plt.close()

    # ✅ -------- PLOT 2: Confusion Matrix --------
    cm = confusion_matrix(y_test, rf_y_pred.round())

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")

    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")
    plt.close()

    # ✅ -------- METRICS --------
    mse = mean_squared_error(y_test, rf_y_pred)
    r2 = r2_score(y_test, rf_y_pred)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2_score", r2)

    print(f"✅ MSE: {mse}, R^2: {r2}")

    # ✅ -------- SAVE MODEL --------
    mlflow.sklearn.log_model(rf_model, "random_forest_model")

    mlflow.log_artifact("models\\annotated_data.csv")

print("✅ All results logged to MLflow UI successfully!")

In [ ]:
# Train a linear regression model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# plot graph of predicted vs actual with a line
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual difficulty")
plt.ylabel("Predicted difficulty")
plt.title("Predicted vs Actual difficulty")
plt.show()


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred.round())
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()



# Calculate MSE and R^2
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse}, R^2: {r2}")

In [ ]:
# Train Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)

plt.figure(figsize=(8, 6))
plt.scatter(y_test, rf_y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual difficulty")
plt.ylabel("Predicted difficulty")
plt.title("Predicted vs Actual difficulty")
plt.show()

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, rf_y_pred.round())
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()



mse = mean_squared_error(y_test, rf_y_pred)
r2 = r2_score(y_test, rf_y_pred)
print(f"MSE: {mse}, R^2: {r2}")

In [ ]:
# SVR Model
from sklearn.svm import SVR
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train, y_train)
svr_y_pred = svr_model.predict(X_test)

# Visualize SVR results
plt.figure(figsize=(8, 6))
plt.scatter(y_test, svr_y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual difficulty")
plt.ylabel("Predicted difficulty")
plt.title("Predicted vs Actual difficulty")
plt.show()

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, svr_y_pred.round())
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

mse = mean_squared_error(y_test, svr_y_pred)
r2 = r2_score(y_test, svr_y_pred)
print(f"MSE: {mse}, R^2: {r2}")

In [ ]:
# Training XGBoost Regressor
from xgboost import XGBRegressor

xg_reg = XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                max_depth=5, alpha=10, n_estimators=100)
xg_reg.fit(X_train, y_train)

# Testing the model
from sklearn.metrics import mean_squared_error, r2_score
preds = xg_reg.predict(X_test)
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")